In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import CategoricalCrossentropy

In [2]:
import tensorflow as tf
import random

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [3]:
data = pd.read_csv("./data/train.csv")
X, x_test, Y, y_test = train_test_split(data.drop(columns=["label"]), data["label"], test_size=0.2, random_state=seed, stratify=data["label"])

In [4]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=seed, stratify=Y)

In [37]:
image_shape = (28, 28, 1)

model = Sequential([
    Reshape(image_shape, name="first_reshape"),
    Conv2D(64, (1, 1), activation="relu", name="second_conv_1"),
    Conv2D(64, (3, 3), activation="relu", name="second_conv_2", padding="same", strides=1),
    Conv2D(256, (1, 1), activation="relu", name="second_conv_3"),
    MaxPooling2D(pool_size=(2, 2), name="second_maxpool"),
    Conv2D(128, (1, 1), activation="relu", name="third_conv_1"),
    Conv2D(128, (3, 3), activation="relu", name="third_conv_2", padding="same", strides=1),
    Conv2D(512, (1, 1), activation="relu", name="third_conv_3"),
    MaxPooling2D(pool_size=(2, 2), name="third_maxpool"),
    Conv2D(256, (1, 1), activation="relu", name="fourth_conv_1"),
    Conv2D(256, (3, 3), activation="relu", name="fourth_conv_2", padding="same", strides=1),
    Conv2D(1024, (1, 1), activation="relu", name="fourth_conv_3"),
    MaxPooling2D(pool_size=(2, 2), name="fourth_maxpool"),
    Conv2D(512, (1, 1), activation="relu", name="fifth_conv_1"),
    Conv2D(512, (3, 3), activation="relu", name="fifth_conv_2", padding="same", strides=1),
    Conv2D(2048, (1, 1), activation="relu", name="fifth_conv_3"),
    AveragePooling2D(pool_size=(2, 2), name="sixth_avgpool"),
    Flatten(name="seven_flatten"),
    Dense(10, activation="softmax", name="seven_dense")
])

In [38]:
model.compile(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(learning_rate=0.005),
    metrics=["accuracy"]
)

In [39]:
early = EarlyStopping(monitor="val_loss", patience=25, restore_best_weights=True)

history = model.fit(x_train, to_categorical(y_train), epochs=100, batch_size=1000, validation_data=[x_val, to_categorical(y_val)], callbacks=[early])

Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.1050 - loss: 33.1950 - val_accuracy: 0.1743 - val_loss: 2.6211
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.1446 - loss: 2.3710 - val_accuracy: 0.3674 - val_loss: 1.8515
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.5123 - loss: 1.4022 - val_accuracy: 0.8082 - val_loss: 0.6102
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.8349 - loss: 0.5137 - val_accuracy: 0.8942 - val_loss: 0.3448
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.9072 - loss: 0.3005 - val_accuracy: 0.9320 - val_loss: 0.2187
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.9327 - loss: 0.2079 - val_accuracy: 0.9424 - val_loss: 0.1803
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.9551 - loss: 0.1392 - val_accuracy: 0.9579 - val_loss: 0.1405
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.9643 - loss: 0.1165 - val_accuracy: 0.9621 - 

KeyboardInterrupt: 

In [31]:
loss, accuracy = model.evaluate(x_test, to_categorical(y_test), verbose=1)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

263/263 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.9843 - loss: 0.0836
Loss: 0.08148768544197083
Accuracy: 0.9823809266090393


In [32]:
y_pred = model.predict(x_test, verbose=1)
y_pred = np.argmax(y_pred, axis=1)
print("Precision: %.3f" % precision_score(y_test, y_pred, average="weighted"))
print("Recall: %.3f" % recall_score(y_test, y_pred, average="weighted"))
print("F1-score: %.3f" % f1_score(y_test, y_pred, average="weighted"))

263/263 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step
Precision: 0.982
Recall: 0.982
F1-score: 0.982


In [8]:
model_file_name = "models/recognizer3.keras"
model.save(model_file_name)